**Caso 5**

# Caso de Estudio: Edificios Inteligentes y Energía

## Contexto del Negocio

En la construcción de edificios sostenibles (como los de certificación LEED), **saber cuánta energía gastarán es la clave del éxito**. No se trata solo de ecología, sino de estrategia:

* **Ahorro:** Menos gasto en recibos de luz y gas a largo plazo.
* **Equipos:** Saber exactamente qué tamaño de aire acondicionado o calefacción comprar.
* **Normas:** Cumplir con las leyes ambientales y reducir la contaminación.

La energía que necesita un edificio para estar a una temperatura agradable depende de su **forma y diseño**: si es muy alto, si tiene muchas ventanas, qué tan grande es el techo o hacia dónde está orientado. Predecir esto **antes de construir** permite elegir el mejor diseño y ahorrar mucho dinero.


## Objetivo del Caso de Estudio

El objetivo es **analizar la eficiencia de los edificios** usando datos de sus características físicas para tomar mejores decisiones.

### Tareas Principales

1. **Predicción de Gasto (Regresión):**
Calcular el número exacto de **carga de calefacción** (`Y1`). Esto nos dice cuánta energía consumirá el edificio para mantenerse caliente.
2. **Etiquetado de Diseño (Clasificación):**
Separar los diseños en dos grupos: **Eficiente (1)** o **No Eficiente (0)**. Si el gasto de energía supera un límite, el diseño se descarta por no ser ahorrador.


In [3]:
import sys

assert sys.version_info >= (3, 7)

In [4]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [5]:
import matplotlib.pyplot as plt

plt.rc('font', size=12)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=12)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [6]:
import numpy as np

np.random.seed(7)

In [7]:
#Instalamos paquetes básicos que vamos a utilizar, por si es que no existieran en el entorno
!pip install -q numpy pandas matplotlib seaborn scikit-learn

print("✅ Paquetes básicos instalados")

✅ Paquetes básicos instalados


In [8]:
# IMPORTACION DE LIBRERIAS
# Sin estas librerías no podemos leer datos ni trabajar con ellos.
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import matplotlib
import sklearn

#verificamos las versiones:

print("📚 Versiones de librerías:")
print(f"  - NumPy: {np.__version__}")
print(f"  - Pandas: {pd.__version__}")
print(f"  - Matplotlib: {matplotlib.__version__}")
print(f"  - Scikit-learn: {sklearn.__version__}")

📚 Versiones de librerías:
  - NumPy: 2.0.2
  - Pandas: 2.2.2
  - Matplotlib: 3.10.0
  - Scikit-learn: 1.6.1


In [9]:
# URL directa del dataset (UCI)
# Descargamos los datos por internet, así todos usan exactamente los mismos datos, sin errores.
url = "https://archive.ics.uci.edu/static/public/242/energy%2Befficiency.zip"

response = requests.get(url)
assert response.status_code==200

In [10]:
# INSPECCION DEL ARCHIVO ZIP.
# Se abre el archivo y se lee el dataset.
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    z.namelist()

In [11]:
# APERTURA DEL ARCHIVO
# Abre el archivo Excel y lo convierte en un dataframe.
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open("ENB2012_data.xlsx") as f:
        df = pd.read_excel(f)

df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [12]:
# REVISION DE LAS FILAS Y COLUMNAS
df.shape
assert df.shape[0]==768 and df.shape[1]==10

In [13]:
# REVISAR EL TIPO DE DATOS CON LOS QUE TRABAJAMOS
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [14]:
# RENOMBRAMOS LAS COLUMNAS
# Cambiamos por sus nombres que vimos en la url
df.columns = [
    "Relative_Compactness",      # X1
    "Surface_Area",              # X2
    "Wall_Area",                 # X3
    "Roof_Area",                 # X4
    "Overall_Height",            # X5
    "Orientation",               # X6
    "Glazing_Area",              # X7
    "Glazing_Area_Distribution", # X8
    "Heating_Load",              # Y1
    "Cooling_Load"               # Y2
]
df.head()

,Relative_Compactness,Surface_Area,Wall_Area,Roof_Area,Overall_Height,Orientation,Glazing_Area,Glazing_Area_Distribution,Heating_Load,Cooling_Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [15]:
#verificamos datos básicos del dataframe
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Relative_Compactness,768.0,0.764167,0.105777,0.62,0.6825,0.75,0.8300,0.98
Surface_Area,768.0,671.708333,88.086116,514.50,606.3750,673.75,741.1250,808.50
Wall_Area,768.0,318.500000,43.626481,245.00,294.0000,318.50,343.0000,416.50
Roof_Area,768.0,176.604167,45.165950,110.25,140.8750,183.75,220.5000,220.50
Overall_Height,768.0,5.250000,1.751140,3.50,3.5000,5.25,7.0000,7.00
Orientation,768.0,3.500000,1.118763,2.00,2.7500,3.50,4.2500,5.00
Glazing_Area,768.0,0.234375,0.133221,0.00,0.1000,0.25,0.4000,0.40
Glazing_Area_Distribution,768.0,2.812500,1.550960,0.00,1.7500,3.00,4.0000,5.00
Heating_Load,768.0,22.307195,10.090204,6.01,12.9925,18.95,31.6675,43.10
Cooling_Load,768.0,24.587760,9.513306,10.90,15.6200,22.08,33.1325,48.03


In [16]:
# FEATURE ENGIENEERING (Overall_Surface)
# Creamos una nueva columna sumando paredes + techo.
df["Overall_Surface"] = df["Wall_Area"] + df["Roof_Area"]

df[["Wall_Area", "Roof_Area", "Overall_Surface"]].head()
# Overall_Surface = Wall_Area + Roof_Area captura la superficie total de intercambio térmico,
# simplificando y clarificando la relación con la carga térmica.

,Wall_Area,Roof_Area,Overall_Surface
0,294.0,110.25,404.25
1,294.0,110.25,404.25
2,294.0,110.25,404.25
3,294.0,110.25,404.25
4,318.5,122.50,441.00


In [17]:
# VERIFICACION Y LIMPIEZA DE FEATURES
# Comparamos Surface_Area con Overall_Surface
# Si son iguales, eliminamos la columna redundante

difference = (df["Surface_Area"] - df["Overall_Surface"]).abs()

if (difference == 0).all():
    df.drop(columns=["Overall_Surface"], inplace=True)
    print("Overall_Surface eliminada (era idéntica a Surface_Area)")
else:
    print(f"Las columnas son diferentes. Diferencia máxima: {difference.max():.2f}")
    print(f"Diferencia mínima: {difference.min():.2f}")


Las columnas son diferentes. Diferencia máxima: 220.50
Diferencia mínima: 110.25


In [18]:
# DEFINIMOS LAS VARIABLES FISICAS
# Elegimos qué columnas se van a usar como entrada del modelo. Esto para decirle al modelo que informacion puede ver
physical_features = [
    "Relative_Compactness",
    "Surface_Area",
    "Wall_Area",
    "Roof_Area",
    "Overall_Height",
    "Orientation",
    "Glazing_Area",
    "Glazing_Area_Distribution",
    "Overall_Surface"
]

target = "Heating_Load"

In [19]:
# CONSTRUCCION DEL PIPELINE
# Creacion de una “receta” automática para preparar los datos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [20]:
# APLICACION DEL PIPELINE
# Escala los datos para que todos estén en el mismo tamaño.
physical_pipeline = Pipeline(
    steps=[
        ("scaler", StandardScaler())
    ]
)

In [21]:
X = df[physical_features]
y = df[target]

X_scaled = physical_pipeline.fit_transform(X)

In [22]:
# VERIFICACION DEL ESCALADO
# Comprobacion de que el escalado funcionó bien.
X_scaled.mean(axis=0)

array([-7.40148683e-17, -4.16333634e-16,  0.00000000e+00,  2.17418676e-16,
        0.00000000e+00,  0.00000000e+00,  1.48029737e-16,  0.00000000e+00,
       -4.07081776e-16])

In [23]:
X_scaled.std(axis=0)

array([1., 1., 1., 1., 1., 1., 1., 1., 1.])